In [1]:
# посмотрим на данные - в этот раз датасет доступен напрямую из sklearn.
# выведем все доступные категории.
# параметр subset отвечает за разделенение данных (train - тренировочная выборка, test - тестовая).

# параметр remove говорит о том, какие части данных нужно удалить, чтобы не допустить переобучения.
# headers - заголовки новостных групп
# quotes - удаление строк, похожих на цитаты из других источников
# footers - удаление блоков из конца текста, похожих на подписи

from sklearn.datasets import fetch_20newsgroups
import numpy as np
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Мы будем использовать только 4 класса текстов: alt.atheism, sci.space, talk.religion.misc, comp.graphics. Используя параметр categories в функции fetch_20newsgroups, задайте список нужных нам категорий и разбейте данные на тренировочную и тестовые части (параметр subset).

Учтите, что сами данные (целевые и нецелевые признаки) лежат в атрибутах data и target

In [2]:
categories = ['alt.atheism','sci.space','talk.religion.misc','comp.graphics']

newgroups_train = fetch_20newsgroups(categories=categories,subset='train',remove=('headers','footers','quotes'))

newgroups_test = fetch_20newsgroups(categories=categories,subset='test',remove=('headers','footers','quotes'))

In [3]:
X_train = newgroups_train.data
y_train = newgroups_train.target
X_test = newgroups_test.data
y_test = newgroups_test.target

In [4]:
print(type(X_train))
print(type(X_train[0]))
print(type(y_train))
print(type(y_train[0]))

<class 'list'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'numpy.int64'>


Выведите на экран по 1 тексту из каждой категории.

In [46]:
for i in range(np.unique(y_train).shape[0]):
    print(np.array(X_train)[y_train==i][0])

I have a request for those who would like to see Charley Wingate
respond to the "Charley Challenges" (and judging from my e-mail, there
appear to be quite a few of you.)  

It is clear that Mr. Wingate intends to continue to post tangential or
unrelated articles while ingoring the Challenges themselves.  Between
the last two re-postings of the Challenges, I noted perhaps a dozen or
more posts by Mr. Wingate, none of which answered a single Challenge.  

It seems unmistakable to me that Mr. Wingate hopes that the questions
will just go away, and he is doing his level best to change the
subject.  Given that this seems a rather common net.theist tactic, I
would like to suggest that we impress upon him our desire for answers,
in the following manner:

1. Ignore any future articles by Mr. Wingate that do not address the
Challenges, until he answers them or explictly announces that he
refuses to do so.

--or--

2. If you must respond to one of his articles, include within it
something simila

проведем небольшой эксперимент по отбору признаков: датасет изкоробочный, специально для обучения машинному обучению (sic), НО...
... проверим данные на наличие пробелов и пустых строк

In [6]:
print(X_train.count(''))
print(X_train.count(' '))
print(X_train.count('  '))

47
4
0


Как мы видим, среди признаков внезапно оказались пустые строки, и этим пустым строкам присвоен класс! Очевидно, что такое недопустимо, поэтому датасет необходимо немного вычистить.

Заметьте, что мы делаем проверку на строку с 1м, 2мя, 3мя пробелами, но не с 5 пробелами и больше. Чтобы эффективно найти строки с некоторым неизвестным числом пробелов, чтоит воспользоваться регулярным выражением ^\\s*$ - данная регулярка срабатывает на строках, состоящих целиком из пробелов (\s - это пробельный символ, квантификатор * указывает, что число повторений такого символа больше одного, символ ^ указывает на начало строки, а знак доллара- на конец строки).

Для нахождения возьмем функцию match из библиотеки re регулярных выражений в питоне.
Доки по функции match. Обратите внимание, что она возвратит None, если паттерн не совпал с заданной строкой, или возвратит некий match object, если будет совпадение.

Задача: в тестовой и тренировочной выборках найти индексы пробельных строк. Зная индексы (это должен быть массив индексов), можно удалить такие элементы из тренировочной и тестовой выборок. Для удаления можете использовать логические маски, или np.delete

In [82]:
print(len(y_train))
print(len(y_test))

2034
1353


In [4]:
import re

pattern = re.compile('^\s*$')

######
### ВАШ КОД

######

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train_fix = np.array(X_train)[np.array([re.match(pattern,i) for i in X_train])==None]

y_train = y_train[np.array([re.match(pattern,i) for i in X_train])==None]

X_test_fix = np.array(X_test)[np.array([re.match(pattern,i) for i in X_test])==None]

y_test = y_test[np.array([re.match(pattern,i) for i in X_test])==None]

Выведем число элементов после очистки, должно получиться 1977 и 1318 элементов в тренировочной и тестовой выборках:

In [5]:
assert len(y_train) == 1977
assert len(X_train_fix) == 1977
assert len(y_test) == 1318
assert len(X_test_fix) == 1318

Посмотрим на то, что из себя представляют целевые признаки. Это целое число, обозначающее индекс категории.
Преобразуем этот индекс в имя категории. Для этого воспользуемся генератором списков и методом target_names, который по индексу вернет нам название категории.

In [6]:
y_train = np.array([newgroups_train.target_names[i] for i in y_train])
y_test = np.array([newgroups_test.target_names[i] for i in y_test])


In [96]:
np.unique(y_train)

array(['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc'],
      dtype='<U18')

Для работы со счетчиками мы возьмем реализацию из библиотеки sklearn.

по ссылке тут можно почитать про сами счетчики (мешок слов и TF-IDF)

Мешок слов
Документация

Можно начать с очень простой идеи. Давайте разобъем все предложения на слова. Составим словарь всех слов, которые будут встречаться во всех наших текстах. И отметим, встречается ли это слово в нашем конкретном примере. Другими словами, пусть в таблице в строках будут предложения, в столбцах - слова, а в ячейках число, которое показывает сколько раз это слово встречалось в этом предложении. Получается, что каждому объекту выборки будет сопоставлен вектор.

Векторизацию мы делаем сразу методом fit_transform - он эквивалентен последовательному вызову

bow = count_vectorizer.fit(data).transform(data)
Очевидно, что метод fit составляет словарь, а transform делает вектор из предложения, согласно имеющемуся словарю.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
texts = [
    "I've been searching for the right words to thank you for this breather.",
    "You have been wonderful and a blessing at all times",
    "I promise i wont take your help for granted and will fulfil my promise."
]
bow = count_vectorizer.fit_transform(texts)
print("Shape=", bow.shape)

Shape= (3, 28)


In [99]:
# посмотрим на словарь всех слов (метод vocabulary_)
# число - это индекс слова в строке матрицы

count_vectorizer.vocabulary_

{'ve': 21,
 'been': 3,
 'searching': 14,
 'for': 6,
 'the': 17,
 'right': 13,
 'words': 25,
 'to': 20,
 'thank': 16,
 'you': 26,
 'this': 18,
 'breather': 5,
 'have': 9,
 'wonderful': 23,
 'and': 1,
 'blessing': 4,
 'at': 2,
 'all': 0,
 'times': 19,
 'promise': 12,
 'wont': 24,
 'take': 15,
 'your': 27,
 'help': 10,
 'granted': 8,
 'will': 22,
 'fulfil': 7,
 'my': 11}

Теперь составим ту самую матрицу, где в столбцах слова, а в строках тексты.

Как мы видим, в первом и втором предложениях есть слово "been", а в третьем его нет (так как у "been" индекс равен 3).
Так как векторайзер возвращает разряженную матрицу, то воспользуемся методом .toarray(), чтобы превратить ее в numpy-массив.

In [101]:
bow.toarray()

array([[0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 1, 0],
       [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 1]], dtype=int64)

При векторизации можно удалить "стоп-слова" - они не несут какого-то смысла, но нужны для грамматики (параметр stop_words). Как мы видим, словарь стал заметно меньше, соответсвенно и вектор тоже стал короче.

In [102]:
count_vectorizer = CountVectorizer(stop_words='english')
bow = count_vectorizer.fit_transform(texts)
print("Shape=", bow.shape)
count_vectorizer.vocabulary_

Shape= (3, 14)


{'ve': 10,
 'searching': 7,
 'right': 6,
 'words': 13,
 'thank': 8,
 'breather': 1,
 'wonderful': 11,
 'blessing': 0,
 'times': 9,
 'promise': 5,
 'wont': 12,
 'help': 4,
 'granted': 3,
 'fulfil': 2}

Мешок слов не учитывает "веса" слов, он просто смотрит их вхождение в документ. Вероятно, было бы полезно взвесить каким-то образом каждое слово в документе. Действительно, если слово встречается во всех документах, то, наверное, его вес небольшой. А если редкое слово встречается в некоторых документах, то скорее всего оно какое-то узко тематическое.

Один из способов взвесить слова - это использовать меру tf-idf, где:

TF - term frequency - частота слова для каждой статьи


IDF - inverse document frequency* — обратная частота документа - уменьшает вес часто встречаемых слов


TF-IDF = TF * IDF

Синтаксис такой же, как и у мешка слов

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
texts = [
    "I've been searching for the right words to thank you for this breather.",
    "You have been wonderful and a blessing at all times",
    "I promise i wont take your help for granted and will fulfil my promise."
]
bow = tfidf_vectorizer.fit_transform(texts)
print("Shape=", bow.shape)

Shape= (3, 14)


In [105]:
tfidf_vectorizer.vocabulary_

{'ve': 10,
 'searching': 7,
 'right': 6,
 'words': 13,
 'thank': 8,
 'breather': 1,
 'wonderful': 11,
 'blessing': 0,
 'times': 9,
 'promise': 5,
 'wont': 12,
 'help': 4,
 'granted': 3,
 'fulfil': 2}

In [106]:
bow.toarray()

array([[0.        , 0.40824829, 0.        , 0.        , 0.        ,
        0.        , 0.40824829, 0.40824829, 0.40824829, 0.        ,
        0.40824829, 0.        , 0.        , 0.40824829],
       [0.57735027, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.57735027,
        0.        , 0.57735027, 0.        , 0.        ],
       [0.        , 0.        , 0.35355339, 0.35355339, 0.35355339,
        0.70710678, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.35355339, 0.        ]])

Обратясь к примерам выше и к документации по ссылкам, создайте векторайзеры для подготовленных в предыдущем параграфе данных. Используйте английские стоп-слова.

Так как у нас есть две части (тренировочная и тестовая выборки), то векторайзер нужно обучить на словах из обоих выборок (подумайте, почему). Так как у нас питон, а наши выборки это массивы строк, то объеденить их очень просто - просто сложить. После того, как вы обучили векторайзер на всех словах, проведите трансформации отдельно для тестовой, и отдельно для тренировочных частей.

Векторизованные части назовите xcv_test/train для count_vectorizer, и xTfidf_test/train для TF-IDF - это нужно для корректной работы тестов и примеров ниже.

In [9]:
vect_cv = CountVectorizer(stop_words='english')
vect_cv.fit(np.array(list(X_train_fix) + list(X_test_fix)))

Xcv_train = vect_cv.transform(X_train_fix)

Xcv_test = vect_cv.transform(X_test_fix)

In [10]:
vect_tf = TfidfVectorizer(stop_words='english')

vect_tf.fit(np.array(list(X_train_fix) + list(X_test_fix)))

XTfidf_train = vect_tf.transform(X_train_fix)

XTfidf_test = vect_tf.transform(X_test_fix)

In [11]:
# count vectorizer
assert Xcv_train.shape == (1977, 33529)
assert Xcv_test.shape == (1318, 33529)

#tf-idf
assert XTfidf_train.shape == (1977, 33529)
assert XTfidf_test.shape == (1318, 33529)

Вспомните статью из блога, которая была в самом начале.

Модель классификатора строится на основе обучающей выборки. По ней необходимо найти следующюю статистику:

 1 Частоты классов в корпусе объектов (сколько объектов принадлежит каждому из классов) (classes_stats)
 2 Cуммарное число слов в документах каждого класса (words_per_class, далее см. )
 3 Частоты слов в пределах каждого класса (word_freqs_per_class, далее используется для расчета )
 4 Размер словаря выборки (число признаков) - кол-во уникальных слов в выборке (num_features)
По сути, это метод fit классификатора

Сигнатура класса:

class NaiveBayes:
    def fit(self, x, y) -> None

    def predict(self, x) -> List[Int]
Для начала отдельно подсчитаем различные статистики, описанные в статье и в материале выше. Так как у нас уже готовы все данные, то считать будем по count_vectorizer'у (то есть xcv_ ...).

Общее число документов в обучающей выборке (doc_num)

In [12]:
doc_num = Xcv_train.shape[0]
# ПРОВЕРКА
assert doc_num == 1977

Словарь, содержащий число объектов каждого класса (classes_stats)

In [13]:
classes_stats = {i:np.sum(y_train == i) for i in np.unique(y_train)}

In [14]:
# ПРОВЕРКА
assert classes_stats['alt.atheism'] == 468
assert classes_stats['comp.graphics'] == 571
assert classes_stats['sci.space'] == 577
assert classes_stats['talk.religion.misc'] == 361

Число уникальных признаков (слов) в тренировочной выборке (num_features)

In [15]:
num_features = Xcv_train.shape[1]
# ПРОВЕРКА
assert num_features == 33529

Создадим словарь indexes, в котором ключом будет являться имя класса, а значением - список строк матрицы X, принадлежащих этому классу. Этот список пригодится нам дальше, так как будет играть роль маски. Для поиска класса каждой из строк используйте целевой вектор.

In [16]:
indexes = {i:[ind for ind,elem in enumerate(y_train==i) if elem] for i in np.unique(y_train)}

In [17]:
# ПРОВЕРКА
# так как в словаре очень много элементов, то проверим случайные элементы из списка.
# если вы все сделали правильно, то эти элементы совпадут.
assert indexes['sci.space'][35] == 111
assert indexes['comp.graphics'][42] == 159
assert indexes['talk.religion.misc'][67] == 312
assert indexes['alt.atheism'][89] == 372

Используя найденные выше индексы, подсчитаем два важных параметра words_per_class и word_freqs_per_class.
Обе этих переменных являются словарями, но первая из них отвечает за суммарное число слов, использованных в каждом классе, а вторая показывает, сколько раз конкретное слово встретилось в документах определенного класса. Соответственно, формат переменной words_per_class - {str: int}, формат word_freqs_per_class - {str: List}. Мы специально объеденили поиск двух разных статистик в одном блоке, чтобы избежать лишних циклов.

Чтобы найти в X строки, относящиеся к тому или иному классу, воспользуйтесь поиском по маске indexes для нужного класса.

Также помните, что X - это разряженная матрица, но из нее можно получить обычный список через метод toarray()

In [18]:
x_arr = Xcv_train.toarray()

words_per_class = {}
word_freqs_per_class = {}

for cls in indexes.keys():
    class_idxs = indexes[cls] # нашли индексы строк матрицы, относящихся к классу cls

    subarray_rows = x_arr[class_idxs] # нашли подмассив, относящийся к  классу cls
    subarray_sum = np.sum(subarray_rows, axis = 0) # провели суммирование по столбцам
    word_freqs_per_class[cls] = subarray_sum

    words_per_class[cls] = len(subarray_sum[subarray_sum != 0]) # узнали,
        # сколько слов было использовано в рамках одного класса, 
        # то есть просто подсчитали число ненулевых элементов

words_per_class, word_freqs_per_class

({'alt.atheism': 8737,
  'comp.graphics': 10592,
  'sci.space': 13273,
  'talk.religion.misc': 8860},
 {'alt.atheism': array([ 0, 17,  0, ...,  0,  0,  0], dtype=int64),
  'comp.graphics': array([26, 12,  0, ...,  0,  0,  2], dtype=int64),
  'sci.space': array([32, 92,  2, ...,  0,  0,  0], dtype=int64),
  'talk.religion.misc': array([1, 9, 0, ..., 0, 0, 0], dtype=int64)})

Все вышенаписанные переменные образуют метод fit будущего классификатора. Теперь внесите этот код в метод fit, и не забудьте сделать найденные переменные полями экземпляра класса при помощи self

In [ ]:
class NaiveBayes:
    def __init__(self):
        pass

    def fit(self, x, y):
        self.doc_num = x.shape[0] # Общее число документов в обучающей выборке (doc_num)
        
        self.classes_stats = {i:np.sum(y == i) for i in np.unique(y)} #Словарь, содержащий число объектов 
        # каждого класса (classes_stats)
        
        self.num_features = x.shape[1] # Число уникальных признаков (слов) в тренировочной выборке (num_features)
        
        self.indexes = {i:[ind for ind,elem in enumerate(y==i) if elem] for i in np.unique(y)} 
        
        self.x_arr = x.toarray()

        self.words_per_class = {} #  суммарное число слов, использованных в каждом классе
        self.word_freqs_per_class = {} # вторая показывает, сколько раз конкретное слово встретилось 
        # в документах определенного класса

        for cls in indexes.keys():
            class_idxs = self.indexes[cls] # нашли индексы строк матрицы, относящихся к классу cls

            subarray_rows = x_arr[class_idxs] # нашли подмассив, относящийся к  классу cls
            subarray_sum = np.sum(subarray_rows, axis = 0) # провели суммирование по столбцам
            self.word_freqs_per_class[cls] = subarray_sum

            self.words_per_class[cls] = len(subarray_sum[subarray_sum != 0]) # узнали,
        # сколько слов было использовано в рамках одного класса, 
        # то есть просто подсчитали число ненулевых элементов

        self.words_per_class, self.word_freqs_per_class

Теперь реализуем метод predict. Вспомните еще раз большую формулу из начала этого раздела. Если вы внимательно читали статью, то заметили, что в примере мы получили не чистые вероятности классов, а всего лишь числовые оценки. Далее эти оценки можно перевести в вероятности, но мы этого делать не будем.

Тогда промежуточный выход классификатора обозначим так:

pred_per_class = {<номер строки в тестовой выборке>: {<класс 1>: <оценка>, ... , <класс n>: <оценка>}}

Таким образом итоговый класс к которому будет отнесена строка - просто класс с наибольшей оценкой

In [99]:
from collections import defaultdict
import math

def predict(x):
    x = x.toarray()
    pred_per_class = defaultdict(dict)
    for i in range(x.shape[0]):
        pred_per_class[i] = {j:math.log(classes_stats[j]/doc_num) + 
                    np.sum(np.log((word_freqs_per_class[j][np.nonzero(x[i,])]+1)/(num_features+words_per_class[j])))
                   for j in np.unique(y_train)}
    pred = []
    for i in range(x.shape[0]):
        pred.append(max(pred_per_class[i],key=pred_per_class[i].get))
    
    return np.array(pred)

In [90]:
# проверим размерность предсказаний.
assert len(predict(Xcv_test)) == len(y_test)

In [100]:
predict(Xcv_test)

array(['sci.space', 'comp.graphics', 'comp.graphics', ..., 'sci.space',
       'comp.graphics', 'comp.graphics'], dtype='<U18')

Теперь соберем весь классфикатор вместе, внеся функцию predict внутрь ранее написанного класса

In [101]:
class NaiveBayes:
    def __init__(self):
        pass

    def fit(self, x, y):
        self.y = y
        self.doc_num = x.shape[0] # Общее число документов в обучающей выборке (doc_num)
        
        self.classes_stats = {i:np.sum(y == i) for i in np.unique(y)} #Словарь, содержащий число объектов 
        # каждого класса (classes_stats)
        
        self.num_features = x.shape[1] # Число уникальных признаков (слов) в тренировочной выборке (num_features)
        
        self.indexes = {i:[ind for ind,elem in enumerate(y==i) if elem] for i in np.unique(y)} 
        
        self.x_arr = x.toarray()

        self.words_per_class = {} #  суммарное число слов, использованных в каждом классе
        self.word_freqs_per_class = {} # вторая показывает, сколько раз конкретное слово встретилось 
        # в документах определенного класса

        for cls in indexes.keys():
            class_idxs = self.indexes[cls] # нашли индексы строк матрицы, относящихся к классу cls

            subarray_rows = x_arr[class_idxs] # нашли подмассив, относящийся к  классу cls
            subarray_sum = np.sum(subarray_rows, axis = 0) # провели суммирование по столбцам
            self.word_freqs_per_class[cls] = subarray_sum

            self.words_per_class[cls] = len(subarray_sum[subarray_sum != 0]) # узнали,
        # сколько слов было использовано в рамках одного класса, 
        # то есть просто подсчитали число ненулевых элементов
    
    def predict(self,x):
        import math
        x = x.toarray()
        pred_per_class = defaultdict(dict)
        for i in range(x.shape[0]):
            pred_per_class[i] = {j:math.log(self.classes_stats[j]/self.doc_num) + 
                    np.sum(np.log((self.word_freqs_per_class[j][np.nonzero(x[i,])]+1)/(self.num_features+self.words_per_class[j])))
                   for j in np.unique(self.y)}
        pred = []
        for i in range(x.shape[0]):
            pred.append(max(pred_per_class[i],key=pred_per_class[i].get))
    
        return np.array(pred)

В sklearn за матрицу ошибок и метрики отвечают confusion_matrix и classification_report.

In [102]:
from sklearn.metrics import classification_report, confusion_matrix

nb_cv = NaiveBayes()

nb_cv.fit(Xcv_train, y_train)
pred = nb_cv.predict(Xcv_test)
        
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))


                    precision    recall  f1-score   support

       alt.atheism       0.62      0.74      0.67       311
     comp.graphics       0.91      0.90      0.90       384
         sci.space       0.77      0.90      0.83       378
talk.religion.misc       0.72      0.37      0.49       245

          accuracy                           0.76      1318
         macro avg       0.76      0.73      0.72      1318
      weighted avg       0.77      0.76      0.75      1318

[[229   9  41  32]
 [  9 344  31   0]
 [ 16  17 342   3]
 [117   8  29  91]]


Проверим классификатор на полученных ранее tf-idf векторах

In [103]:
nb_tf = NaiveBayes()

nb_tf.fit(XTfidf_train, y_train)
pred = nb_tf.predict(XTfidf_test)
        
print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))

                    precision    recall  f1-score   support

       alt.atheism       0.62      0.74      0.67       311
     comp.graphics       0.91      0.90      0.90       384
         sci.space       0.77      0.90      0.83       378
talk.religion.misc       0.72      0.37      0.49       245

          accuracy                           0.76      1318
         macro avg       0.76      0.73      0.72      1318
      weighted avg       0.77      0.76      0.75      1318

[[229   9  41  32]
 [  9 344  31   0]
 [ 16  17 342   3]
 [117   8  29  91]]


Сравним с версией из sklearn

In [104]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=4)
clf.fit(Xcv_train, y_train)

y_pred = clf.predict(Xcv_test)
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

                    precision    recall  f1-score   support

       alt.atheism       0.62      0.75      0.68       311
     comp.graphics       0.91      0.91      0.91       384
         sci.space       0.83      0.89      0.86       378
talk.religion.misc       0.68      0.42      0.52       245

          accuracy                           0.78      1318
         macro avg       0.76      0.74      0.74      1318
      weighted avg       0.78      0.78      0.77      1318

[[232   9  28  42]
 [ 12 351  20   1]
 [ 20  18 335   5]
 [112   9  20 104]]


In [105]:
clf = MultinomialNB(alpha=4)
clf.fit(XTfidf_train, y_train)

y_pred = clf.predict(XTfidf_test)
print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))

                    precision    recall  f1-score   support

       alt.atheism       0.60      0.64      0.62       311
     comp.graphics       0.84      0.93      0.88       384
         sci.space       0.63      0.92      0.75       378
talk.religion.misc       0.85      0.04      0.09       245

          accuracy                           0.69      1318
         macro avg       0.73      0.63      0.58      1318
      weighted avg       0.73      0.69      0.63      1318

[[199  22  88   2]
 [  1 356  27   0]
 [  5  24 349   0]
 [126  20  88  11]]


Как вы заметили, качество предсказаний неоднородно (какие-то классы определяются хорошо, какие-то не очень хорошо). В чем может быть причина этого?

Самостоятельно попробуйте найти топ-10 наиболее часто встречающихся слов в каждой категории, посмотрите на эти слова и напишите свои идеи, почему же при классификации точность падает на той или иной категории.

In [131]:
ind = np.sort(np.sum(Xcv_train.toarray()[y_train=='sci.space'],0))[-1:-11:-1]
count = np.sum(Xcv_train.toarray()[y_train=='sci.space'],0)

indx = []

for i in range(count.shape[0]):
    if count[i] in ind:
        indx.append(i)


In [132]:
indx

[9480, 11183, 17495, 18144, 18499, 20809, 21918, 27466, 28096, 30302]

In [151]:
res = []
for k,i in vect_cv.vocabulary_.items():
    if i in indx:
        res.append(k)

In [152]:
res

['like',
 'just',
 'data',
 'earth',
 'space',
 'time',
 'orbit',
 'launch',
 'nasa',
 'shuttle']